In [2]:
import pandas as pd

In [7]:
df = pd.read_csv('glassdoor_jobs.csv')
# shape of datasets
print("1. Shape of datasets are : ",df.shape)
print("Number of rows: ",df.shape[0])
print("Number of columns: ",df.shape[1])

1. Shape of datasets are :  (956, 15)
Number of rows:  956
Number of columns:  15


In [27]:
# datatypes and list columns
print("Data Types for each column:")
print(df.dtypes)
category_dt_columns = df.select_dtypes(include=['object','category']).columns
numeric_dt_columns = df.select_dtypes(include=['int64','float64']).columns
print("")
print('Category Datatype Columns: ',list(category_dt_columns))
print("")
print('Numeric Datatype Columns: ',list(numeric_dt_columns))

Data Types for each column:
Unnamed: 0             int64
Job Title             object
Salary Estimate       object
Job Description       object
Rating               float64
Company Name          object
Location              object
Headquarters          object
Size                  object
Founded                int64
Type of ownership     object
Industry              object
Sector                object
Revenue               object
Competitors           object
dtype: object

Category Datatype Columns:  ['Job Title', 'Salary Estimate', 'Job Description', 'Company Name', 'Location', 'Headquarters', 'Size', 'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors']

Numeric Datatype Columns:  ['Unnamed: 0', 'Rating', 'Founded']


In [33]:
print("Handle Missing column in  datasets:")
print(df.isnull().any().any())

Handle Missing column in  datasets:
False
